# TKO chimera processing

In [1]:
knitr::opts_knit$set(root.dir = here::here())

In [ ]:
setwd(here::here())

In [8]:
library("metacell")
library("tgstat")
library("tgconfig")
library("Matrix")
library("dplyr")

In [28]:
scdb_init("scrna_db/", force_reinit = T)

initializing scdb to scrna_db/



In [29]:
mat <- scdb_mat("tko_chim")

#### Remove cells belonging to E8.5 chimera embryos. Only use chimera embryos from E7.5

In [5]:
e85_embryos <- paste0("TKO29_", c(13:19))
ig_cells <- colnames(mat@mat)[mat@cell_metadata[colnames(mat@mat), "embryo"] %in% e85_embryos]
ig_cells <- c(mat@ignore_cells, ig_cells)

mat <- scm_ignore_cells(scmat = mat, ig_cells = ig_cells)

#### Remove extraembryonic ectoderm and parietal endoderm cells

In [6]:
source("scripts/pipeline/remove_exe_ectoderm_and_parietal_endo_cls.R")
mat <- remove_exe_ectoderm_and_parietal_endoderm(mat_query = mat)
scdb_add_mat(id = "tko_chim", mat = mat)

#### Merge cells with the wildtype atlas

In [7]:
source("scripts/pipeline/merge_umi_mat_with_wt10_umi_mat.R")
mat_new <- merge_umi_mat_with_wt10(scmat = mat)
scdb_add_mat(id = "tko_chim_wt10", mat = mat_new)

#### Generate single-cell balanced kNN-graph
This step needs the file  
_data/tet_tko.bad_genes.txt_

In [23]:
source("scripts/pipeline/gen_cgraph.R")
gen_cgraph(mat_nm = "tko_chim_wt10", force_recompute_cgraph = F)

initializing scdb to scrna_db/



[1] TRUE

#### Transfer cell type annotation to chimera cells

In [19]:
source("scripts/pipeline/transfer_cell_type_annotation.R")
transfer_color_chimera_tetraploid(mat_nm = "tko_chim_wt10")

#### Time chimera embryos based on their host and KO cells

In [20]:
source("scripts/pipeline/tko_chim_timing.R")
source("scripts/pipeline/transfer_time_annotation.R")
tko_chim_timing()
# tko_chimera_plot_cor_time_dist_per_emb(mat_nm = "tko_chim_wt10")

#### Export infered time and cell type into table

In [39]:
source("scripts/pipeline/create_metadata.R")
create_tko_chimera_metadata()

#### Generate a separate UMI matrix and metacell object for the TKO cells

In [30]:
source("scripts/pipeline/generat_ko_mat.R")
generate_ko_mat_chimera()

In [31]:
source("scripts/pipeline/generic_mc.R")
tgconfig::override_params(config_file = "config/tet_chim.yaml", package = "metacell")

bad_genes <- read.table("data/tet_tko.bad_genes.txt", sep = "\t", stringsAsFactors = F)
bad_genes <- bad_genes[, 1]

mat_nm <- "tko_chim_ko"

generate_mc(name = mat_nm, add_bad_genes = bad_genes)

will downsample the matrix, N= 2222

will build balanced knn graph on 6984 cells and 425 genes, this can be a bit heavy for >20,000 cells

running bootstrap to generate cocluster

done resampling

filtered 3131824 left with 469000 based on co-cluster imbalance

building metacell object, #mc 87

add batch counts

compute footprints

compute absolute ps

compute coverage ps

reordering metacells by hclust and most variable two markers

reorder on AK041755;Cdh1 vs Phlda2

starting split outliers 

add batch counts

compute footprints

compute absolute ps

compute coverage ps



In [32]:
# Run the line below if you want to transfer cell type annotation from the WT atlas to the tko_chim_ko metacell object

# wt_atlas <- mcell_gen_atlas(
#     mat_id = "sing_emb_wt10",
#     mc_id = "sing_emb_wt10_recolored",
#     gset_id = "sing_emb_wt10",
#     mc2d_id = "sing_emb_wt10_recolored"
# )

# cmp <- mcell_proj_on_atlas(
#     mat_id = "tko_chim_ko",
#     mc_id = "tko_chim_ko",
#     atlas = wt_atlas,
#     fig_cmp_dir = paste0("figs/atlas_projection_", "tko_chim_ko"), recolor_mc_id = "tko_chim_ko", ten2mars = F
# )

In [36]:
tgconfig::set_param(param = "mcell_mc2d_T_edge", value = 0.03, package = "metacell")
mcell_mc2d_force_knn(mc2d_id = "tko_chim_ko", mc_id = "tko_chim_ko", graph_id = "tko_chim_ko", graph_parametric = T, feats_gset = "tko_chim_ko")
mcell_mc2d_plot("tko_chim_ko")

got 74 feat genes for mc graph construction



png 
  2